In [1]:
import theano
from theano import tensor as T
import numpy as np
from collections import OrderedDict
from tqdm import tqdm
from bnr_ml.utils.helpers import *
import time
from lasagne import layers
from lasagne.updates import rmsprop
from itertools import tee

ERROR (theano.sandbox.cuda): ERROR: Not using GPU. Initialisation of device gpu failed:
initCnmem: cnmemInit call failed! Reason=CNMEM_STATUS_OUT_OF_MEMORY. numdev=1

/Library/Python/2.7/site-packages/theano/tensor/signal/downsample.py:6: UserWarning: downsample module has been moved to the theano.tensor.signal.pool module.
  "downsample module has been moved to the theano.tensor.signal.pool module.")
/Users/colingaudreau/anaconda2/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
def _get_cost_optim_multi(output, truth, S, B, C,lmbda_coord=5., lmbda_noobj=0.5, lmbda_obj=3., iou_thresh=1e-3):
    '''
    Calculates cost for multiple objects in a scene without for loops or scan (so reduces the amount of variable
    created in the theano computation graph).  A cell is associated with a certain object if the iou of that cell
    and the object is higher than any other ground truth object. and the rest of the objectness scores are pushed
    towards zero.
    '''

    # calculate height/width of individual cell
    block_height, block_width = 1. / S[0], 1./ S[1]

    # get the offset of each cell
    offset_x, offset_y = meshgrid2D(T.arange(0,1,block_width), T.arange(0,1,block_height))

    # get indices for x,y,w,h,object-ness for easy access
    x_idx, y_idx = T.arange(0,5*B,5), T.arange(1,5*B, 5)
    w_idx, h_idx = T.arange(2,5*B,5), T.arange(3,5*B,5)
    conf_idx = T.arange(4,5*B,5)

    # Get position predictions with offsets.
    pred_x = (output[:,x_idx] + offset_x.dimshuffle('x','x',0,1)).dimshuffle(0,'x',1,2,3)
    pred_y = (output[:,y_idx] + offset_y.dimshuffle('x','x',0,1)).dimshuffle(0,'x',1,2,3)
    pred_w, pred_h = output[:,w_idx].dimshuffle(0,'x',1,2,3), output[:,h_idx].dimshuffle(0,'x',1,2,3)
    pred_conf = output[:,conf_idx].dimshuffle(0,'x',1,2,3)
    pred_class = output[:,-C:].dimshuffle(0,'x',1,2,3)

    pred_w, pred_h = T.maximum(pred_w, 0.), T.maximum(pred_h, 0.)

    x_idx, y_idx = T.arange(0,truth.shape[1],4+C), T.arange(1,truth.shape[1],4+C)
    w_idx, h_idx = T.arange(2,truth.shape[1],4+C), T.arange(3,truth.shape[1],4+C)
    class_idx,_ = theano.scan(
        lambda x: T.arange(x,x+C,1),
        sequences = T.arange(4,truth.shape[1],4+C)
    )

    truth_x, truth_y = truth[:,x_idx], truth[:,y_idx]
    truth_w, truth_h = truth[:,w_idx], truth[:,h_idx]
    truth_class = truth[:, class_idx]


    # Get intersection region bounding box coordinates
    xi = T.maximum(pred_x, truth_x.dimshuffle(0,1,'x','x','x'))
    xf = T.minimum(pred_x + pred_w, (truth_x + truth_w).dimshuffle(0,1,'x','x','x'))
    yi = T.maximum(pred_y, truth_y.dimshuffle(0,1,'x','x','x'))
    yf = T.minimum(pred_y + pred_h, (truth_y + truth_h).dimshuffle(0,1,'x','x','x'))
    w, h = T.maximum(xf - xi, 0.), T.maximum(yf - yi, 0.)

    # Calculate iou score for predicted boxes and truth
    isec = w * h
    union = (pred_w * pred_h) + (truth_w * truth_h).dimshuffle(0,1,'x','x','x') - isec
    iou = T.maximum(isec/union, 0.)

    # Get index matrix representing max along the 1st dimension for the iou score (reps 'responsible' box).
    maxval_idx, _ = meshgrid2D(T.arange(B), T.arange(truth.shape[0]))
    maxval_idx = maxval_idx.dimshuffle(0,'x',1,'x','x')
    maxval_idx = T.repeat(T.repeat(maxval_idx,S[0],3),S[1],4)

    box_is_resp = T.eq(maxval_idx, iou.argmax(axis=2).dimshuffle(0,1,'x',2,3))

    # Get matrix for the width/height of each cell
    width, height = T.ones(S) / S[1], T.ones(S) / S[0]
    width, height = width.dimshuffle('x','x',0,1), height.dimshuffle('x','x',0,1)
    offset_x, offset_y = offset_x.dimshuffle('x','x',0,1), offset_y.dimshuffle('x','x',0,1)

    # Get bounding box for intersection between CELL and ground truth box.
    xi = T.maximum(offset_x, truth_x.dimshuffle(0,1,'x','x'))
    xf = T.minimum(offset_x + width, (truth_x + truth_w).dimshuffle(0,1,'x','x'))
    yi = T.maximum(offset_y, truth_y.dimshuffle(0,1,'x','x'))
    yf = T.minimum(offset_y + height, (truth_y + truth_h).dimshuffle(0,1,'x','x'))
    w, h = T.maximum(xf - xi, 0.), T.maximum(yf - yi, 0.)

    # Calculate iou score for the cell.
    isec = w * h
    union = (width * height) + (truth_w* truth_h).dimshuffle(0,1,'x','x') - isec
    iou_cell = T.maximum(isec/union, 0.).dimshuffle(0,1,'x',2,3) # * (np.prod(S)) # normalize the iou to make more sense

    maxval_idx, _ = meshgrid2D(T.arange(iou_cell.shape[1]), T.arange(iou_cell.shape[0]))
    maxval_idx = maxval_idx.dimshuffle(0,1,'x','x','x')
    maxval_idx = T.repeat(T.repeat(T.repeat(maxval_idx, B, 2), S[0], 3), S[1], 4)

    obj_for_cell = T.eq(maxval_idx, iou_cell.argmax(axis=1).dimshuffle(0,'x',1,2,3))

    # Get logical matrix representing minimum iou score for cell to be considered overlapping ground truth.
    cell_intersects = (iou_cell > iou_thresh)

    obj_in_cell_and_resp = T.bitwise_and(T.bitwise_and(cell_intersects, box_is_resp), obj_for_cell)
    conf_is_zero = T.bitwise_and(
        bitwise_not(T.bitwise_and(cell_intersects, box_is_resp)),
        obj_for_cell
    )
    conf_is_zero = conf_is_zero.sum(axis=1, keepdims=True)

    # repeat "cell overlaps" logical matrix for the number of classes.
    pred_class = T.repeat(pred_class, truth.shape[1] // (4 + C), axis=1)

    # repeat the ground truth for class probabilities for each cell.
    truth_class_rep = T.repeat(T.repeat(truth_class.dimshuffle(0,1,2,'x','x'), S[0], axis=3), S[1], axis=4)

    cost = T.sum((pred_conf - iou)[obj_in_cell_and_resp.nonzero()]**2) + \
        lmbda_noobj * T.sum((pred_conf[conf_is_zero.nonzero()])**2) + \
        lmbda_coord * T.sum((pred_x - truth_x.dimshuffle(0,1,'x','x','x'))[obj_in_cell_and_resp.nonzero()]**2) + \
        lmbda_coord * T.sum((pred_y - truth_y.dimshuffle(0,1,'x','x','x'))[obj_in_cell_and_resp.nonzero()]**2) + \
        lmbda_coord * T.sum((pred_w.sqrt() - truth_w.dimshuffle(0,1,'x','x','x').sqrt())[obj_in_cell_and_resp.nonzero()]**2) + \
        lmbda_coord * T.sum((pred_h.sqrt() - truth_h.dimshuffle(0,1,'x','x','x').sqrt())[obj_in_cell_and_resp.nonzero()]**2) + \
        lmbda_obj * T.sum(((pred_class - truth_class_rep)[cell_intersects.nonzero()])**2)
    pdb.set_trace()	
    return cost / T.maximum(1., truth.shape[0])

In [26]:
S = (4,4)
B = 2
C = 4

In [27]:
truth = np.asarray([0.25,0.25,0.5,0.5,1.,0.,0.,0.])

output = np.asarray([0.25,0.25,0.5,0.5,1.]).reshape((1,-1,1,1))
output = np.repeat(np.repeat(output, S[0], axis=2), S[1], axis=3)

offx, offy = np.meshgrid(np.arange(0,1,1./S[1]), np.arange(0,1,1./S[0]))

offx, offy = offx.reshape((1,1) + offx.shape), offy.reshape((1,1) + offy.shape)

output[:,[0]] -= offx
output[:,[1]] -= offy

output = np.concatenate((output, np.zeros((1,5 * B,S[0],S[0]))), axis=1)

pred = np.asarray([1.,0.,0.,0.]).reshape((1,-1,1,1))
pred = np.repeat(np.repeat(pred, S[0], axis=2), S[1], axis=3)

output = np.concatenate((output, pred), axis=1)
truth = truth.reshape((1,-1))

In [28]:
X, y = theano.shared(output), theano.shared(truth)

In [31]:
cost = _get_cost_optim_multi(X, y, S, B, C)

> <ipython-input-2-4aa3e6c29b69>(101)_get_cost_optim_multi()
-> return cost / T.maximum(1., truth.shape[0])
(Pdb) output[0,0].eval()
array([[ 0.25,  0.  , -0.25, -0.5 ],
       [ 0.25,  0.  , -0.25, -0.5 ],
       [ 0.25,  0.  , -0.25, -0.5 ],
       [ 0.25,  0.  , -0.25, -0.5 ]])
(Pdb) truth_x.shape.eval()
array([1, 1])
(Pdb) pred_x.shape.eval()
array([1, 1, 2, 4, 4])
(Pdb) pred_x[0,0,0].eval()
array([[ 0.25,  0.25,  0.25,  0.25],
       [ 0.25,  0.25,  0.25,  0.25],
       [ 0.25,  0.25,  0.25,  0.25],
       [ 0.25,  0.25,  0.25,  0.25]])
(Pdb) pred_y[0,0,0].eval()
array([[ 0.25,  0.25,  0.25,  0.25],
       [ 0.25,  0.25,  0.25,  0.25],
       [ 0.25,  0.25,  0.25,  0.25],
       [ 0.25,  0.25,  0.25,  0.25]])
(Pdb) pred_w[0,0,0].eval()
array([[ 0.5,  0.5,  0.5,  0.5],
       [ 0.5,  0.5,  0.5,  0.5],
       [ 0.5,  0.5,  0.5,  0.5],
       [ 0.5,  0.5,  0.5,  0.5]])
(Pdb) pred_h[0,0,0].eval()
array([[ 0.5,  0.5,  0.5,  0.5],
       [ 0.5,  0.5,  0.5,  0.5],
       [ 0.5,  0.5,  0.

In [49]:
x = T.as_tensor(4.0)

In [50]:
y = x**2

In [51]:
z = T.copy(y)

In [53]:
z.eval()

MissingInputError: ('Undeclared input', <TensorType(float32, scalar)>)

In [52]:
T.grad(z, x).eval()

ValueError: Elemwise{pow,no_inplace}.0 is not in list